In [1]:
from nltk.corpus import stopwords
import numpy as np
import json
import glob
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy

import pyLDAvis
import pyLDAvis.gensim

# Data preparation

In [2]:

def load_data(file):
    with open(file, 'r',encoding="utf-8") as f:
        data=json.load(f)
    return data

def write_data(file,data):
    with open(file,"w",encoding="utf-8") as f:
        json.dump(data,f,indent=4)


In [3]:
stopWordsItalian=stopwords.words("italian")
stopWordsItalian

['ad',
 'al',
 'allo',
 'ai',
 'agli',
 'all',
 'agl',
 'alla',
 'alle',
 'con',
 'col',
 'coi',
 'da',
 'dal',
 'dallo',
 'dai',
 'dagli',
 'dall',
 'dagl',
 'dalla',
 'dalle',
 'di',
 'del',
 'dello',
 'dei',
 'degli',
 'dell',
 'degl',
 'della',
 'delle',
 'in',
 'nel',
 'nello',
 'nei',
 'negli',
 'nell',
 'negl',
 'nella',
 'nelle',
 'su',
 'sul',
 'sullo',
 'sui',
 'sugli',
 'sull',
 'sugl',
 'sulla',
 'sulle',
 'per',
 'tra',
 'contro',
 'io',
 'tu',
 'lui',
 'lei',
 'noi',
 'voi',
 'loro',
 'mio',
 'mia',
 'miei',
 'mie',
 'tuo',
 'tua',
 'tuoi',
 'tue',
 'suo',
 'sua',
 'suoi',
 'sue',
 'nostro',
 'nostra',
 'nostri',
 'nostre',
 'vostro',
 'vostra',
 'vostri',
 'vostre',
 'mi',
 'ti',
 'ci',
 'vi',
 'lo',
 'la',
 'li',
 'le',
 'gli',
 'ne',
 'il',
 'un',
 'uno',
 'una',
 'ma',
 'ed',
 'se',
 'perché',
 'anche',
 'come',
 'dov',
 'dove',
 'che',
 'chi',
 'cui',
 'non',
 'più',
 'quale',
 'quanto',
 'quanti',
 'quanta',
 'quante',
 'quello',
 'quelli',
 'quella',
 'quelle',
 'q

In [4]:
def get_list_of_docs(data):
    out=[]
    for k in data.keys():
        current_docs=data[k]
        
        out.extend(current_docs)
    return out


In [5]:
data=load_data("..\index_parsed.json")
data=get_list_of_docs(data)

data


<>:1: DeprecationWarning: invalid escape sequence \i
<>:1: DeprecationWarning: invalid escape sequence \i
C:\Users\Wissel\AppData\Local\Temp\ipykernel_28148\2751208739.py:1: DeprecationWarning: invalid escape sequence \i
  data=load_data("..\index_parsed.json")


['',
 '',
 ' linguaggio grammatiche linguaggi grammatiche linguaggi informatica •ubiquitari applicazioni –linguaggi programmazione •compilatori interpreti –linguaggi comunicazione •protocolli dialogo traentità omologhe –linguaggi intefacce •specifica sequenze operazioni •paradigmatici teoria –molti importanti problemi teorici riconducibili dell’appartenenza stringa linguaggio ',
 ' linguaggio grammatiche tre approcci diversi •approccio insiemistico –utile determinare proprietà elementari linguaggi •approccio generativo –grammatiche formali •approccio riconoscitivo –automi riconoscitori concetti matematici base •insiemi •relazioni •funzioni ',
 ' linguaggio grammatiche insiemi •consideriamo insiemi finiti insiemi infiniti •a cardinalità dell’insieme finito •alcuni insiemi infiniti numeri n naturali contiene zero n naturali positivi z interi relativi zinteri positivi z interi negativiq razionali relativi q razionali positivi q razionali negativi r reali r reali positivi r reali negativi 

In [6]:
#lemmatization
#usiamo Noun, aggettivi, verbi, avverbi
def lemmatization(texts,allowed_posttags=["NOUN","ADJ","VERB","ADV"]):
    nlp = spacy.load("it_core_news_sm",disable=["parser","ner"])              #spcay non fa parsing e ner (possiamo vedere la differenza utilizzando ner)

    text_out=[]
    for text in texts:
        
        doc=nlp(text)           
        new_text=[]
        #per ogni token nel doc prendo solo le parole che sono etichettate con i tag specificati
        #se un token ha quel tag, allora prendo il suo lemma
        for token in doc:
            if token.pos_ in allowed_posttags:
                new_text.append(token.lemma_)
        final=" ".join(new_text)

        text_out.append(final)
    return text_out


In [7]:
lem_text=lemmatization(data)
print("Esempio frase lemmizzata")
print(lem_text[5][0:90])

Esempio frase lemmizzata
linguaggio grammatico parte pal’ sottoinsieme dire parte indicare pa finito allora pa oper


In [8]:
#preprocessing con gensim
def gen_words(texts):
    final=[]
    for text in texts:
        new=gensim.utils.simple_preprocess(text,deacc=True) #toglie gli accenti
        final.append(new)
    return final




In [9]:
data_words=gen_words(lem_text)

print(data_words[5][0:90])

['linguaggio', 'grammatico', 'parte', 'pal', 'sottoinsieme', 'dire', 'parte', 'indicare', 'pa', 'finito', 'allora', 'pa', 'operazione', 'insiemi', 'unione', 'finire', 'ab', 'commutativa', 'associativo', 'intersezione', 'finire', 'mina', 'commutativa', 'associativo', 'intersezione', 'distributivo', 'rispetto', 'partizione', 'insieme', 'sottoinsieme', 'tale', 'an', 'ai', 'aj']


In [32]:
#BIGRAMS and TRIGRAMS

#iperparametri threshold, count etx...
bigrams_phrases=gensim.models.Phrases(data_words,min_count=2,threshold=100)       #min_count=numero minimo di occorrenze di un ngram per essere definito ngram
trigram_phrases=gensim.models.Phrases(bigrams_phrases[data_words],threshold=100)

bigram=gensim.models.phrases.Phraser(bigrams_phrases)
trigrams=gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return [bigram[doc] for doc in texts]

def make_trigrams(texts):
    return [trigrams[bigram[doc]] for doc in texts]


data_bigrams=make_bigrams(data_words)               #bigrammi
data_trigrams_bigrams=make_trigrams(data_bigrams)   #bigrammi e trigrammi


print(data_trigrams_bigrams[9])         #i bigrammi_trigrammi vengono suddivisi da un _


['linguaggio', 'grammatico', 'esempio', 'relazionere', 'equivalenza', 'consideriamo', 'relazione', 'esistere', 'rk', 'tali', 'nqkr', 'relazione', 'equivalenza', 'le', 'classe', 'classe', 'resto', 'divisione', 'operazione', 'relazione', 'unione', 'xy', 'xy', 'chiusuro', 'transitivo', 'yn', 'yny', 'tali', 'yiyi', 'chiusura', 'transitivo', 'riflessivo', 'rr', 'xx']


In [47]:
#TF-IDF con Gensim
from gensim.models import TfidfModel

#creazione vettori
id2words=corpora.Dictionary(data_trigrams_bigrams)

texts=data_trigrams_bigrams

corpus=[id2words.doc2bow(text) for text in texts ]      #bag of words


#tfidf
tfidf=TfidfModel(corpus,id2word=id2words)
'''low_value=0.002
words=[]
words_missing_in_tfidf=[]
#Il metodo permette di lavorare su un corpus composto solo da token ritenuti rilevanti (con tfidf> di una soglia )
for i in range(0,len(corpus)):
    bow=corpus[i]       #bagofword associato alla parola
    
    low_value_words=[]
    tfidf_ids=[id for id in tfidf[bow]]
    
    bow_ids=[id for id, value in bow]
    print("bow ids",bow_ids)
    low_value_words=[id for id, value in tfidf[bow] if value<low_value]     #elimino se il tf-idf della parola è sotto 0.03
    print("sotto soglia",low_value_words)
    drops=low_value_words+words_missing_in_tfidf
    print("drops",drops)
    for item in drops:
        words.append(id2words[item])
    
    words_missing_in_tfidf=[id for id in bow_ids if id not in tfidf_ids]    

    new_bow=[b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    print("new bow",new_bow)
    print("original bow",bow)
    corpus[i]=new_bow
    

'''
print(corpus[20])


[(5, 1), (11, 1), (93, 1), (106, 2), (213, 2), (220, 2), (230, 2), (232, 2), (233, 2), (246, 1), (247, 1), (248, 1), (249, 1)]


In [22]:
id2words=corpora.Dictionary(data_words)

corpus=[]
for text in data_words:
    new=id2words.doc2bow(text)
    corpus.append(new)


print(corpus[5][0:90])      #genera una lista di tuple: (indice, frequenza)

[(5, 1), (11, 1), (13, 1), (45, 1), (49, 2), (50, 1), (52, 1), (67, 2), (71, 1), (72, 1), (73, 1), (74, 1), (75, 2), (76, 2), (77, 1), (78, 1), (79, 1), (80, 1), (81, 2), (82, 1), (83, 2), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [48]:
#LDA model
lda_model=gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2words, num_topics=30, random_state=100, update_every=1, chunksize=100, passes=10, alpha="auto")

In [24]:
#visualizzazione dinamica
pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim.prepare(lda_model,corpus,id2words,mds="mmds",R=30)
vis

c:\Users\Wissel\OneDrive\Desktop\università\MAGISTRALE\SII\Progetto_SII\LILL_AI\project\.venv\lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
29    -0.291662  0.043317       1        1  18.557219
1      0.178337 -0.310439       2        1   8.314793
9      0.249103  0.105376       3        1   8.046839
4      0.226747  0.210204       4        1   7.270814
16     0.314028 -0.107015       5        1   7.069668
3      0.005555 -0.419318       6        1   6.355427
10    -0.058572  0.346632       7        1   5.863034
22     0.333481 -0.017968       8        1   5.612326
6     -0.275694 -0.107934       9        1   4.675187
28    -0.221657 -0.020143      10        1   3.913847
8     -0.257514  0.190665      11        1   3.822735
12    -0.177389 -0.258248      12        1   3.629403
18    -0.164497 -0.141887      13        1   2.993256
23     0.077291  0.242593      14        1   2.084539
0     -0.026835  0.207287      15        1   2.064643
17     0.109231 -0.183616      16        1   1.964709
11     0.039893 -0.156618      17        1   1.543173
5     -0.127859  0.167574      18        1   1.415726
19     0.010383 -0.135705      19        1   1.235128
14     0.000844  0.147823      20        1   1.137281
24     0.105570  0.022306      21        1   0.982938
26    -0.054902  0.085433      22        1   0.809479
13     0.000634  0.011145      23        1   0.080375
25     0.000973  0.011185      24        1   0.080357
20     0.000752  0.011225      25        1   0.079517
21     0.000752  0.011225      26        1   0.079517
7      0.000752  0.011225      27        1   0.079517
27     0.000752  0.011225      28        1   0.079517
2      0.000752  0.011225      29        1   0.079517
15     0.000752  0.011225      30        1   0.079517, topic_info=            Term       Freq      Total Category  logprob  loglift
355         tipo  33.000000  33.000000  Default  30.0000  30.0000
40     relazione  23.000000  23.000000  Default  29.0000  29.0000
213  espressione  27.000000  27.000000  Default  28.0000  28.0000
58        numero  12.000000  12.000000  Default  27.0000  27.0000
220     regolare  27.000000  27.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
25    ubiquitari   0.002384   1.404497  Topic30  -6.4151   0.7584
26     approccio   0.002384   4.842892  Topic30  -6.4151  -0.4794
27        automa   0.002384   2.248591  Topic30  -6.4151   0.2878
28          base   0.002384   5.955580  Topic30  -6.4151  -0.6863
29      concetto   0.002384   2.163477  Topic30  -6.4151   0.3263

[1529 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
113      11  0.750290         aa
526      12  0.760389        aaa
184       9  0.737085       aaaa
356       5  0.865352      aaaab
357       5  0.699867     aaaabb
...     ...       ...        ...
129      11  0.750280         yx
130      11  0.943377         yz
70        1  0.685491    zintero
601       7  0.728933  אdenotare
602       7  0.728933      אsono

[893 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[30, 2, 10, 5, 17, 4, 11, 23, 7, 29, 9, 13, 19, 24, 1, 18, 12, 6, 20, 15, 25, 27, 14, 26, 21, 22, 8, 28, 3, 16])

In [49]:
#visualizzazione dinamica
pyLDAvis.enable_notebook()
vis_tfidf=pyLDAvis.gensim.prepare(lda_model,corpus,id2words,mds="mmds",R=30)
vis_tfidf

c:\Users\Wissel\OneDrive\Desktop\università\MAGISTRALE\SII\Progetto_SII\LILL_AI\project\.venv\lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
23     0.342024  0.091126       1        1  15.707548
12     0.305221 -0.272802       2        1   9.907805
13    -0.262119  0.197117       3        1   7.039090
14    -0.112219  0.273392       4        1   6.542795
25     0.076336 -0.299945       5        1   6.315109
9      0.261511 -0.132713       6        1   6.111436
18     0.038531  0.334359       7        1   5.328475
5     -0.083470 -0.288404       8        1   4.875921
22    -0.250181  0.067206       9        1   3.781419
3      0.187398  0.195751      10        1   3.740721
0      0.271980  0.008577      11        1   3.702120
19     0.018468  0.241177      12        1   3.372378
26    -0.263308 -0.015809      13        1   3.010303
17    -0.186360  0.122788      14        1   2.810093
7      0.114296 -0.188484      15        1   2.559161
4     -0.210776 -0.072243      16        1   2.551503
6     -0.237973 -0.160062      17        1   2.503131
11    -0.139080 -0.137098      18        1   2.101771
24     0.049002  0.157459      19        1   1.726999
20     0.106055  0.119651      20        1   1.698128
27    -0.065783 -0.159787      21        1   1.417470
8      0.095045 -0.084913      22        1   1.191822
10    -0.038729  0.121336      23        1   1.189256
1      0.017600 -0.042896      24        1   0.303534
15    -0.004973 -0.013078      25        1   0.086805
16    -0.005574 -0.012661      26        1   0.086133
2     -0.005508 -0.012410      27        1   0.085996
28    -0.005688 -0.012128      28        1   0.085240
21    -0.005750 -0.012178      29        1   0.084370
29    -0.005974 -0.012325      30        1   0.083468, topic_info=            Term      Freq      Total Category  logprob  loglift
213  espressione  27.00000  27.000000  Default  30.0000  30.0000
353         tipo  35.00000  35.000000  Default  29.0000  29.0000
220     regolare  27.00000  27.000000  Default  28.0000  28.0000
233    esercizio  20.00000  20.000000  Default  27.0000  27.0000
40     relazione  25.00000  25.000000  Default  26.0000  26.0000
..           ...       ...        ...      ...      ...      ...
25    ubiquitari   0.00242   1.419413  Topic30  -6.4036   0.7141
26     approccio   0.00242   4.969769  Topic30  -6.4036  -0.5390
27        automa   0.00242   2.426148  Topic30  -6.4036   0.1781
28          base   0.00242   6.646317  Topic30  -6.4036  -0.8297
29      concetto   0.00242   2.341752  Topic30  -6.4036   0.2135

[1463 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
113       6  0.706918            aa
518       4  0.704517           aaa
184       9  0.741301          aaaa
354       1  0.824276         aaaab
357       1  0.895803  aaaabbbbcccc
...     ...       ...           ...
129       6  0.706725            yx
130       3  0.869448            yz
70        7  0.710131       zintero
594      10  0.742467     אdenotare
595      10  0.742467         אsono

[975 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[24, 13, 14, 15, 26, 10, 19, 6, 23, 4, 1, 20, 27, 18, 8, 5, 7, 12, 25, 21, 28, 9, 11, 2, 16, 17, 3, 29, 22, 30])